In [111]:
# implement https://arxiv.org/pdf/1705.08690.pdf on avalanche framework on permuted MNIST
# https://aahaanmaini.medium.com/mimicking-human-continual-learning-in-a-neural-network-c15e1ae11d70
#continual learning

## create naive and deepgen models

In [39]:
from torch import nn
from avalanche.models import SimpleMLP

#Naive approach
model_naive = SimpleMLP(num_classes=10, hidden_size = 400, hidden_layers=2)

#Continual Learning approach
model_cl_scholar = SimpleMLP(num_classes=10, hidden_size = 400, hidden_layers=2)


## create GAN

In [101]:
#https://github.com/znxlwm/pytorch-MNIST-CelebA-GAN-DCGAN/blob/master/pytorch_MNIST_GAN.py
import matplotlib.pyplot as plt
import itertools
import pickle
#import imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# G(z)
class generator(nn.Module):
    # initializers
    def __init__(self, input_size=32, n_class = 10):
        super(generator, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, 512)
        self.fc3 = nn.Linear(self.fc2.out_features, 1024)
        self.fc4 = nn.Linear(self.fc3.out_features, n_class)

    # forward method
    def forward(self, input):
        x = F.leaky_relu(self.fc1(input), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.tanh(self.fc4(x))

        return x

class discriminator(nn.Module):
    # initializers
    def __init__(self, input_size=32, n_class=10):
        super(discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, 512)
        self.fc3 = nn.Linear(self.fc2.out_features, 256)
        self.fc4 = nn.Linear(self.fc3.out_features, n_class)

    # forward method
    def forward(self, input):
        x = F.leaky_relu(self.fc1(input), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.sigmoid(self.fc4(x))

        return x

fixed_z_ = torch.randn((5 * 5, 100))    # fixed noise
fixed_z_ = Variable(fixed_z_, volatile=True)
"""
def show_result(num_epoch, show = False, save = False, path = 'result.png', isFix=False):
    z_ = torch.randn((5*5, 100))
    z_ = Variable(z_.cuda(), volatile=True)

    G.eval()
    if isFix:
        test_images = G(fixed_z_)
    else:
        test_images = G(z_)
    G.train()

    size_figure_grid = 5
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(5, 5))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)

    for k in range(5*5):
        i = k // 5
        j = k % 5
        ax[i, j].cla()
        ax[i, j].imshow(test_images[k, :].cpu().data.view(28, 28).numpy(), cmap='gray')

    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')
    plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['D_losses']))

    y1 = hist['D_losses']
    y2 = hist['G_losses']

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()
"""
# training parameters
batch_size = 128
lr = 0.0002
train_epoch = 100

# data_loader
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
"""
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transform),
    batch_size=batch_size, shuffle=True)
"""
# network
G = generator(input_size=100, n_class=28*28)
D = discriminator(input_size=28*28, n_class=1)
G
D

# Binary Cross Entropy loss
BCE_loss = nn.BCELoss()

# Adam optimizer
G_optimizer = optim.Adam(G.parameters(), lr=lr)
D_optimizer = optim.Adam(D.parameters(), lr=lr)


c:\Users\mathi\Anaconda3\envs\DL2\lib\site-packages\ipykernel_launcher.py:54: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


In [149]:
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.models import SimpleMLP

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
# create transform to grayscale
transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
])
# CL Benchmark Creation
BM = SplitMNIST(n_experiences=5, seed=12345, train_transform=transform, eval_transform=transform)


In [112]:
# data_loader
"""
# Load data
train_set = BM.train_stream[0].dataset
test_set = BM.test_stream[0].dataset
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

"""

In [159]:
for experience in BM.train_stream:
    x = experience.dataset
print(x[0])

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [161]:

train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
for epoch in range(train_epoch):
    D_losses = []
    G_losses = []
    for experience in BM.train_stream:
        # train discriminator D
        x_ = experience.dataset
        D.zero_grad()
        
        x_ = x_.view(-1, 28 * 28)
        
        mini_batch = x_.size()[0]

        y_real_ = torch.ones(mini_batch)
        y_fake_ = torch.zeros(mini_batch)

        x_, y_real_, y_fake_ = Variable(x_.cuda()), Variable(y_real_.cuda()), Variable(y_fake_.cuda())
        D_result = D(x_)
        D_real_loss = BCE_loss(D_result, y_real_)
        D_real_score = D_result

        z_ = torch.randn((mini_batch, 100))
        z_ = Variable(z_)
        G_result = G(z_)

        D_result = D(G_result)
        D_fake_loss = BCE_loss(D_result, y_fake_)
        D_fake_score = D_result

        D_train_loss = D_real_loss + D_fake_loss

        D_train_loss.backward()
        D_optimizer.step()

        D_losses.append(D_train_loss.data[0])

        # train generator G
        G.zero_grad()

        z_ = torch.randn((mini_batch, 100))
        y_ = torch.ones(mini_batch)

        z_, y_ = Variable(z_.cuda()), Variable(y_.cuda())
        G_result = G(z_)
        D_result = D(G_result)
        G_train_loss = BCE_loss(D_result, y_)
        G_train_loss.backward()
        G_optimizer.step()

        G_losses.append(G_train_loss.data[0])

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
        (epoch + 1), train_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))
    p = 'MNIST_GAN_results/Random_results/MNIST_GAN_' + str(epoch + 1) + '.png'
    fixed_p = 'MNIST_GAN_results/Fixed_results/MNIST_GAN_' + str(epoch + 1) + '.png'
    show_result((epoch+1), save=True, path=p, isFix=False)
    show_result((epoch+1), save=True, path=fixed_p, isFix=True)
    train_hist['D_losses'].append(torch.mean(torch.FloatTensor(D_losses)))
    train_hist['G_losses'].append(torch.mean(torch.FloatTensor(G_losses)))



AttributeError: 'AvalancheSubset' object has no attribute 'reshape'

In [16]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.training.supervised import Naive
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics

optimizer = SGD(model_naive.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()


# Continual learning strategy
cl_strategy_naive = Naive(
    model_naive, optimizer, criterion, train_mb_size=32, train_epochs=2, 
    eval_mb_size=32)

# model_cl_scholar strategy
cl_strategy_scholar = Naive(
    model_cl_scholar, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32)

In [36]:
#train naive model on train stream
#train GAN on 50% train stream data and 50% generated data
#train CL on 50% GAN generated data and 50% train stream data

results = []
tasks_learned_so_far = []

# iterating over the train stream
for experience in BM.train_stream:
    print('Tasks learned so far:', experience.classes_seen_so_far)    

    #train naive model on train stream
    cl_strategy_naive.train(experience, verbose=0)
    #test naive model on test stream
    naive_evaluation = cl_strategy_naive.eval(BM.test_stream)

    #train GAN on 50% train stream data and 50% generated data
    #train CL on 50% GAN generated data and 50% train stream data
    if experience.task_label == 0:
        #train gan only first task
        gan.fit(experience, verbose=0)
        combined_data = experience
    else:
        new_data = gan.generate()
        combined_data = np.concatenate((experience, new_data), axis=0)
    cl_strategy_scholar.train(combined_data, verbose=0)

Tasks learned so far: [0, 7]
-- >> Start of training phase << --
662it [01:52,  5.86it/s]                         
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0616
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9829
100%|██████████| 381/381 [00:08<00:00, 42.95it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0149
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9951
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 63/63 [00:01<00:00, 52.28it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0136
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9960
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 66/66 [00:01<00:00, 44.13it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 1.0057
	Top1_Ac

In [18]:
results

[{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9931079750574335,
  'Loss_Epoch/train_phase/train_stream/Task000': 0.025623612313078668,
  'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9945219123505976,
  'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.01716251609693256,
  'Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001': 0.1054,
  'Loss_Exp/eval_phase/test_stream/Task001/Exp001': 3.032204005432129,
  'Top1_Acc_Exp/eval_phase/test_stream/Task002/Exp002': 0.095,
  'Loss_Exp/eval_phase/test_stream/Task002/Exp002': 3.314279815673828,
  'Top1_Acc_Exp/eval_phase/test_stream/Task003/Exp003': 0.0869,
  'Loss_Exp/eval_phase/test_stream/Task003/Exp003': 3.098995807647705,
  'Top1_Acc_Exp/eval_phase/test_stream/Task004/Exp004': 0.1006,
  'Loss_Exp/eval_phase/test_stream/Task004/Exp004': 3.020626127624512,
  'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.9945219123505976,
  'Top1_Acc_Stream/eval_phase/test_stream/Task001': 0.1054,
  'Top1_Acc_Stream/eval_phase/test_s